In [9]:
## Import relevant libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
import string

In [10]:
def process(tweet):
    # removes links, usernames, twitter special word 'RT' (retweet) and emoticons
    p = re.compile('(http://[^\s]+|@[^\s]*|RT|(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$))|&\w*')
    return p.sub("", tweet)

In [11]:
def sanitize(text):
    # removes punctuation character
    return text.translate(None, string.punctuation)

In [ ]:
download("averaged_perceptron_tagger")
text = word_tokenize("And now for something !!don't I'm completely different")
res = nltk.pos_tag(text)

In [55]:
[(k,v) for (k,v) in res if k not in  string.punctuation ]

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('do', 'VBP'),
 ("n't", 'RB'),
 ('I', 'PRP'),
 ("'m", 'VBP'),
 ('completely', 'RB'),
 ('different', 'JJ')]